# Grover's algorithm

Grover's algorithm is one of the most famous quantum algorithm. Grover's algorithm is a very powerful and useful algorithm in a wide range of fiealds. For the details of Grover's algorithm, please see [Grover's Algorithm](https://qiskit.org/textbook/ch-algorithms/grover.html) in the Qiskit textbook.

Qiskit Aqua has a special class named `Grover` for Grover's algorithm. `Grover` can specify various problem configurations with its parameters such as oracles, state preparation circuits, the number of the iterations, and so on.

## Constructing quantum circuits for Grover's algorithm
A quantum circuit for Grover's algorithm mainly consists of two parts. One is an oracle and the other one is state preparation (including diffusion). 

### Oracle
For an oracle, `Grover` has an `oracle` argument. `QuantumCircuit`, `Oracle`, and `StateVector` can be passed as `oracle`. All the following examples are when $|11\rangle$ is a state that we would like to find. Those state are called `good_state` in `Grover`. 

Note that `Oracle` has 3 subclasses. They are `LogicalExpressionOracle`, `TrutheTableOracle`, and `CustomCircuitOracle`. Each of them aims for different usecases. For example, `LogicalExpressionOracle` is expecially useful for solving 3-SAT problems (see [Using Grover's Algorithm to Perform Quantum Search](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/optimization/grover.ipynb) in Qiskit-community-tutorials).

In [22]:
from qiskit import QuantumCircuit
from qiskit.aqua.algorithms import Grover

#`QuantumCircuit` as the `oracle` argument
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'])
print(grover._grover_operator.oracle)

        
q_0: ─■─
      │ 
q_1: ─■─
        


Note that, in the following `LogicalExpressionOracle` example, `LogicalExpressionOracle` uses an ancilla bit to flip the phase of the good state by using the phase kickback ([Phase Kickback](https://qiskit.org/textbook/ch-gates/phase-kickback.html)). You don't need to pass `good_state` in `Grover` since `Oracle` itsself has `evaluate_classically()` for checking whether an answer is correct or not.

In [21]:
from qiskit.aqua.components.oracles import LogicalExpressionOracle

# `Oracle` (`LogicalExpressionOracle`) as the `oracle` argument
expression = '(a & b)'
oracle = LogicalExpressionOracle(expression)
grover = Grover(oracle=oracle)
print(grover._grover_operator.oracle)

                              
q_0: ────────────■────────────
                 │            
q_1: ────────────■────────────
     ┌───┐┌───┐┌─┴─┐┌───┐┌───┐
q_2: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘


In [8]:
from qiskit.quantum_info import Statevector
oracle = Statevector.from_label('11')
grover = Grover(oracle=oracle, good_state=['11'])
print(grover._grover_operator.oracle)


     ┌─────────────┐              
q_0: ┤0            ├──────────────
     │  UCRZ(0,pi) │┌────────────┐
q_1: ┤1            ├┤ UCRZ(pi/2) ├
     └─────────────┘└────────────┘


### State preparation
A `state_preparation` argument is used to specify a quantum circuit that prepares a quantum state before applying an oracle. By specifying `state_preparation`, it is possible to reduce the search space of Grover's search for more efficient search. By default, a circuit with $H^{\otimes n} $ is used to prepare uniform superposition. The diffusion circuit of Grover's algorithm relrects `state_preparation` automatically.




In [33]:
# Default `state_preparation`
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'])
print(grover._grover_operator.state_preparation)

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘


In [34]:
# Diffusion operator
print(grover._grover_operator)

            ┌───┐┌───┐          ┌───┐┌───┐     
state_0: ─■─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
          │ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
state_1: ─■─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
            └───┘└───┘└───┘└───┘└───┘└───┘└───┘


In [35]:
import numpy as np

# Specifying `state_preparation` 
# to preparet a superposition of |01>, |10>, and |11>
oracle = QuantumCircuit(2)
oracle.cz(0, 1)

theta = 2 * np.arccos(1 / np.sqrt(3))
state_preparation = QuantumCircuit(2)
state_preparation.ry(theta, 0)
state_preparation.ch(0,1)
state_preparation.x(1)
grover = Grover(oracle=oracle, state_preparation=state_preparation,good_state=['11'])
print(grover._grover_operator.state_preparation)

     ┌────────────┐          
q_0: ┤ RY(1.9106) ├──■───────
     └────────────┘┌─┴─┐┌───┐
q_1: ──────────────┤ H ├┤ X ├
                   └───┘└───┘


In [36]:
# Diffusion operator
print(grover._grover_operator)


                      ┌─────────────┐┌───┐     ┌───┐┌────────────┐          
state_0: ─■────────■──┤ RY(-1.9106) ├┤ X ├──■──┤ X ├┤ RY(1.9106) ├──■───────
          │ ┌───┐┌─┴─┐└────┬───┬────┘├───┤┌─┴─┐├───┤└───┬───┬────┘┌─┴─┐┌───┐
state_1: ─■─┤ X ├┤ H ├─────┤ X ├─────┤ H ├┤ X ├┤ H ├────┤ X ├─────┤ H ├┤ X ├
            └───┘└───┘     └───┘     └───┘└───┘└───┘    └───┘     └───┘└───┘


Fore more complicated oracles, state preparation, and options (e.g. `mcx_mode`), we can also pass `GroverOperator` to `Grover`

In [41]:
from qiskit.circuit.library import GroverOperator
oracle = QuantumCircuit(2)
oracle.cz(0,1)
state_preparation = QuantumCircuit(2)
theta = 2 * np.arccos(1 / np.sqrt(3))
state_preparation = QuantumCircuit(2)
state_preparation.ry(theta, 0)
state_preparation.ch(0,1)
state_preparation.x(1)
grover_op = GroverOperator(oracle=oracle, state_preparation=state_preparation, insert_barriers=True, mcx_mode='noancilla')

grover=Grover(oracle=grover_op.oracle, grover_operator=grover_op)
print(grover._grover_operator)


             ░           ┌─────────────┐ ░ ┌───┐          ┌───┐      ░ »
state_0: ─■──░────────■──┤ RY(-1.9106) ├─░─┤ X ├───────■──┤ X ├──────░─»
          │  ░ ┌───┐┌─┴─┐└─────────────┘ ░ ├───┤┌───┐┌─┴─┐├───┤┌───┐ ░ »
state_1: ─■──░─┤ X ├┤ H ├────────────────░─┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├─░─»
             ░ └───┘└───┘                ░ └───┘└───┘└───┘└───┘└───┘ ░ »
«         ┌────────────┐          
«state_0: ┤ RY(1.9106) ├──■───────
«         └────────────┘┌─┴─┐┌───┐
«state_1: ──────────────┤ H ├┤ X ├
«                       └───┘└───┘


## Specifying `good_state`
`good_state` is used to check whether a measument result is correct or not internally. It can be a list of binary strings, `Statevector`, and Callable. 




In [44]:
# list good state
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
good_state = ['11', '00']
grover = Grover(oracle=oracle, good_state=good_state)
print(grover.is_good_state('11'))


True


In [45]:
# `Statevector` good state
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
good_state = Statevector.from_label('11')
grover = Grover(oracle=oracle, good_state=good_state)
print(grover.is_good_state('11'))

True


In [47]:
# Callable good state
def callable_good_state(bitstr):
    if bitstr == "11":
        return True
    else:
        return False

oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=callable_good_state)
print(grover.is_good_state('11'))

True


## The repetition of the Grover's algortihm
The number of repetition of applying the grover operator is important to obtain the correct result with Grover's algortihm. The number of iteration can be set by the `iteration` argument of `Grover`. `iteration` can be an integer or a list of integers. If `iteration` is an integer, `Grover` repeat applying its grover operator the number of `iteration` times. If `iteration` is a list of integer, `Grover` repeat applying its grover operator the number of `iteration` times in order. When `sample_from_iterations` is `True`, a randome integer between 0 and the specified number in `iteration` is used as the repetation number of grover iteration. For more details of the algorithm using `sample_from_iterations`, see "
L. K. Grover (1996), A fast quantum mechanical algorithm for database search,
            [arXiv:quant-ph/9605043](https://arxiv.org/abs/quant-ph/9605043)"


